In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize as op
import scipy.integrate as integ
from CoolProp import AbstractState
from CoolProp.CoolProp import PhaseSI, PropsSI, get_global_param_string
import CoolProp.CoolProp as CoolProp
from CoolProp.HumidAirProp import HAPropsSI

In [2]:
#initial condition: Temp, diameter, mass flow rate
Tempi=42.2
d=0.06
dotm=7.04
Area=(np.pi*d**2)/4

In [3]:
def Pdown(L):
    rho=811
    g=9.81
    deltaP=rho*L*g/(10**6)
    uavg=4.014
    f=0.02807
    ML=(0.5*(uavg**2)*f*rho*(L/d))/1000000
    Pdown=(25+deltaP-ML)*1000000
    return Pdown

In [4]:
def Pup(L):
    rho=351
    g=9.81
    deltaP=rho*L*g/(10**6)
    uavg=5.01
    f=0.02806
    ML=(0.5*(uavg**2)*f*rho*(L/d))/1000000
    Pup=(37.6+deltaP-ML)*100000    
    return Pup

In [5]:
def rho(Temp,P):
    rho=PropsSI("D", "P", P, "T",Temp, "CO2")
    return rho

In [7]:
def Cv(Temp,P):
    C=PropsSI("C", "P", P, "T",Temp, "CO2")
    return C

In [8]:
def k(Temp,P):
    k=PropsSI("L", "P", P, "T",Temp, "CO2")
    return k

In [9]:
def kv(Temp,P):
    C=PropsSI("V", "P", P, "T",Temp, "CO2")
    return C

In [10]:
def Pr(Temp,P):
    Pr=PropsSI("PRANDTL", "P", P, "T",Temp, "CO2")
    return Pr

In [11]:
#The following part is heat transfer through well of downward flow.

In [12]:
#equilvalent h derived from Nussel number
#variable 'hconcrete' is the thermal conductivity of concrete 
def hdown(T,L):
    P=Pdown(L)
    hconcrete=0.5
    V=dotm/(Area*rho(T,P))
    Re=V*d/kv(T,P)
    if Re>4:
        Nu=0.911*Re**0.385*Pr(T,P)**(1/3)
    else:
        Nu=0.989*Re**0.330*Pr(T,P)**(1/3)
    h=Nu*k(T,P)/d
    h1=1/(1/h+0.04/hconcrete)
    return h1

In [13]:
def heattransfer(L, T):
    """
    dotm*Cp*dTm=h*(Ts-Tm)*dAs
    dAs=pi*d*dL
    dTm/dL=h*(Ts-Tm)/(dotm*Cp)
    """
    P=Pdown(L)
    dTdL = np.pi*d*hdown(T,L)*(273+15+0.065*L-T)/dotm/Cv(T,P)
    return dTdL

In [14]:
T0=np.array([38+273])
L0=(223-15)/0.065
sol = integ.solve_ivp(lambda L, T: heattransfer(L, T), (0, 3200), T0,vectorized=True, atol=1e-12, rtol=1e-12)
#Temp from the last iteration is the Temp @ 3200m.
T1=sol.y[0,-1]
print(T1-273)

68.07890658950942


In [15]:
# The following part is heat transfer 

In [16]:
#The equation from PPT
dotm=dotm
P=(20/np.pi)**(1/2)*2*np.pi #parimeter
h2=15 #heat transfer coefficient
Cp2=1900 #Specific heat
length=70 #formation length(check this because it's 112m in PPT)
SurfTemp=223+273 #surface Temp
InletTemp=T1 # Output Temp from the downward flow
T2=SurfTemp-(SurfTemp-InletTemp)*np.exp(-P*length*h2/(dotm*Cp2))
print(T2-273)

178.3680440150456


In [17]:
# The following part is heat transfer through of upward flow

In [18]:
def hup(T,L):
    P=Pup(L)
    hconcrete=0.5
    V=dotm/(Area*rho(T,P))
    Re=V*d/kv(T,P)
    if Re>4:
        Nu=0.911*Re**0.385*Pr(T,P)**(1/3)
    else:
        Nu=0.989*Re**0.330*Pr(T,P)**(1/3)
    h=Nu*k(T,P)/d
    h1=1/(1/h+0.04/hconcrete)
    return h1

In [19]:
def heattransfer1(L, T):
    """
    Similar approach with the downward flow. The difference is now the ambient Temp decrease linearly wrt pipe length.
    """
    dTdL = np.pi*d*hup(T,L)*(273+223-0.065*L-T)/dotm/Cv(T,P)
    return dTdL

In [20]:
#Use Temp out of the saline formation as initial condation
T2=np.array([T2])
L0=(223-15)/0.065
sol1 = integ.solve_ivp(lambda L, T: heattransfer1(L, T), (0, 3200), T2,vectorized=True, atol=1e-12, rtol=1e-12)
T3=sol1.y[0,-1]
print(T3-273)

136.6188851896278
